<a href="https://colab.research.google.com/github/vanfotis/git-code-playground/blob/main/3210213_ML2025_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Εργασία στο μάθημα Μηχανικής Μάθησης 2025-2026
# ΕΥΑΓΓΕΛΟΣ ΦΩΤΗΣ - 3210213

## Multiclass Classification – Glass Identification Dataset

Στόχος της παρούσας εργασίας είναι η υλοποίηση και εφαρμογή βασικών αλγορίθμων
Μηχανικής Μάθησης σε ένα πρόβλημα πολυταξινόμησης (multiclass classification),
σύμφωνα με την εκφώνηση του μαθήματος.

Ως dataset επιλέγεται το **Glass Identification dataset** από το UCI Machine Learning Repository.
Το dataset περιλαμβάνει χημικές μετρήσεις γυαλιού και στόχος είναι η πρόβλεψη
του τύπου γυαλιού, ο οποίος αντιστοιχεί σε διαφορετικές κλάσεις.

Στην εργασία υλοποιούνται από το μηδέν οι ακόλουθοι αλγόριθμοι:
- Principal Component Analysis (PCA)
- Least Squares για classification
- Logistic Regression με Stochastic Gradient Descent
- K Nearest Neighbors (KNN)
- Naive Bayes με κανονικές κατανομές
- Multilayer Perceptron (MLP) μέσω PyTorch
- Support Vector Machines (One-vs-Rest)
- K-Means clustering

Για κάθε μέθοδο παρουσιάζονται αποτελέσματα τόσο στο σύνολο εκπαίδευσης
όσο και στο σύνολο ελέγχου.


In [1]:
# Λήψη του Glass Identification dataset από το UCI Repository
# Το αρχείο θα αποθηκευτεί τοπικά στο περιβάλλον του Colab

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data


--2025-12-26 17:43:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘glass.data’

glass.data              [ <=>                ]  11.62K  --.-KB/s    in 0s      

2025-12-26 17:43:30 (180 MB/s) - ‘glass.data’ saved [11903]



In [2]:
# Έλεγχος ότι το αρχείο κατέβηκε σωστά
!head glass.data


1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.00,1
2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.00,1
3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.00,1
4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.00,1
5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.00,1
6,1.51596,12.79,3.61,1.62,72.97,0.64,8.07,0.00,0.26,1
7,1.51743,13.30,3.60,1.14,73.09,0.58,8.17,0.00,0.00,1
8,1.51756,13.15,3.61,1.05,73.24,0.57,8.24,0.00,0.00,1
9,1.51918,14.04,3.58,1.37,72.08,0.56,8.30,0.00,0.00,1
10,1.51755,13.00,3.60,1.36,72.99,0.57,8.40,0.00,0.11,1


## Περιγραφή του Glass Identification Dataset

Το Glass Identification dataset προέρχεται από το UCI Machine Learning Repository
και περιλαμβάνει μετρήσεις χημικής σύστασης γυαλιού.
Κάθε δείγμα αντιστοιχεί σε ένα θραύσμα γυαλιού και περιγράφεται μέσω
εννέα αριθμητικών χαρακτηριστικών.

Τα χαρακτηριστικά αφορούν το δείκτη διάθλασης και τη συγκέντρωση
διαφόρων χημικών στοιχείων (π.χ. Na, Mg, Al, Si, Ca κ.ά.).
Η μεταβλητή-στόχος (label) αντιστοιχεί στον τύπο του γυαλιού
και το πρόβλημα αντιμετωπίζεται ως πολυταξινόμηση (multiclass classification).

Στη συνέχεια παρουσιάζεται το πλήθος των δειγμάτων, των χαρακτηριστικών
και των διαφορετικών κλάσεων του dataset.


In [3]:
import numpy as np
import pandas as pd

# Ονόματα στηλών σύμφωνα με την περιγραφή του dataset
columns = [
    "Id", "RI", "Na", "Mg", "Al", "Si",
    "K", "Ca", "Ba", "Fe", "Type"
]

# Φόρτωση δεδομένων
df = pd.read_csv("glass.data", header=None, names=columns)

# Εμφάνιση των πρώτων γραμμών
df.head()


,Id,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [4]:
# Διαστάσεις dataset
print("Διαστάσεις dataset:", df.shape)

# Έλεγχος για κενές τιμές
print("\nΚενές τιμές ανά στήλη:")
print(df.isna().sum())

# Πλήθος δειγμάτων ανά κλάση
print("\nΚατανομή κλάσεων:")
print(df["Type"].value_counts().sort_index())


Διαστάσεις dataset: (214, 11)

Κενές τιμές ανά στήλη:
Id      0
RI      0
Na      0
Mg      0
Al      0
Si      0
K       0
Ca      0
Ba      0
Fe      0
Type    0
dtype: int64

Κατανομή κλάσεων:
Type
1    70
2    76
3    17
5    13
6     9
7    29
Name: count, dtype: int64
